# Практическое задание 3 

# Классификация предложений с использованием BERT

## курс "Математические методы анализа текстов"


### ФИО: Садиев Абдурахмон Абдужалолович

## Введение

### Постановка задачи

В этом задании вы будете классифицировать предложения из медицинских статей на несколько классов (background, objective и т.д.). 
Для того, чтобы улучшить качество решения вам предлагается дообучить предобученную нейросетевую архитектуру BERT.

### Библиотеки

Для этого задания вам понадобятся следующие библиотеки:
 - [Pytorch](https://pytorch.org/).
 - [Transformers](https://github.com/huggingface/transformers).
 
### Данные

Скачать данные можно здесь: [ссылка на google диск](https://drive.google.com/file/d/13HlWH8jnmsxqDKrEptxOXQg9kkuQMmGq/view?usp=sharing)

## Часть 1. Подготовка данных

Мы будем работать с предложениями из медицинских статей, разбитых на несколько классов. 

In [0]:
import re
from collections import Counter

Путь к папке с данными:

In [2]:
#colab
from google.colab import drive, files
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#colab
DATA_PATH = "/content/gdrive/My Drive/Bert/sentence_classification_data"
# DATA_PATH = "sentence_classification_data"

Функция считывания данных:

In [0]:
def read_data(file_name):
    """
    Parameters
    ----------
    file_name : str
        Pubmed sentences file path
        
    Returns
    -------
    text_data : list of str
        List of sentences for algorithm
    
    target_data : list of str
        List of sentence categories
    """
    text_data = []
    target_data = []

    with open(file_name, 'r') as f_input:
        for line in f_input:
            if line.startswith('#') or line == '\n':
                continue
            target, text = line.split('\t')[:2]    

            text_data.append(text)
            target_data.append(target)
    
    return text_data, target_data

Считывание данных:

In [0]:
train_data, train_target = read_data(f'{DATA_PATH}/data_train.txt')
test_data, test_target = read_data(f'{DATA_PATH}/data_test.txt')
dev_data, dev_target = read_data(f'{DATA_PATH}/data_dev.txt')


## Часть 2. Построение бейзлайна (1 балл)

В этой части задания вам необходимо построить бейзлайн модель, с которой вы будете сравнивать ваше решение. В качестве бейзлайна вам предлагается использовать модель логистической регрессии на tf-idf представлениях.

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

Перед тем как подать в модель предложения, необходимо их предобработать:
    
1. привести все предложения к нижнему регистру
2. удалить из предложений все непробельные символы кроме букв, цифр
3. все цифры заменить на нули

Метки ответов необходимо преобразовать из текстового вида в числовой (это можно сделать с помощью LabelEncoder).

Затем необходимо построить tf-idf матрицу по выбранным предложениям (используйте для подсчёта tf-idf только train_data!) и обучить на них модель логистической регрессии. Используйте dev выборку для подбора гиперпараметров модели. Добейтесь того, что на test и dev выборках accuracy будет будет выше 0.8.

In [7]:
print(re.sub(r'[^a-zA-Z0-9\s]', '', train_data[5]).lower().replace('\n', ''))

train_data[5]

plasma samples were obtained and analysed with timeresolved immunofluorometric assays determining the plasma levels of map44  masp1  and masp3 


'Plasma samples were obtained and analysed with time-resolved immunofluorometric assays determining the plasma levels of MAp44 , MASP-1 , and MASP-3 .\n'

In [0]:
def preprocess(data):
    data_preprocessed = []
    for sentence in data:
        sentence = re.sub(r'[^a-zA-Z0-9\s]', '', sentence).lower().replace('\n', '')
        sentence = re.sub(r'([0-9])', r'0',sentence)
        data_preprocessed.append(sentence)
    return data_preprocessed
LE = LabelEncoder()

train_data_preprocessed = preprocess(train_data)
y_train = LE.fit_transform(train_target)

test_data_preprocessed = preprocess(test_data)
y_test = LE.transform(test_target)

dev_data_preprocessed = preprocess(dev_data)
y_dev = LE.transform(dev_target)

vectorizer = TfidfVectorizer(ngram_range = (1, 3))
X_train = vectorizer.fit_transform(train_data_preprocessed)
X_test = vectorizer.transform(test_data_preprocessed)
X_dev = vectorizer.transform(dev_data_preprocessed)


In [9]:
import numpy as np

acc_dev =[]
for i, C_reg in enumerate(np.exp(np.linspace(0,6,10))):
    clf = LogisticRegression(penalty = 'l2', 
                              C = C_reg, 
                              solver = 'newton-cg', 
                              multi_class = 'multinomial', 
                              max_iter = 1000)

    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_dev)
    acc = accuracy_score(y_predict, y_dev)
    acc_dev.append(acc)
    print('({0}) C: {1},  Accuracy: {2}\n'.format(i+1, C_reg, acc))




(1) C: 1.0,  Accuracy: 0.7960735517765796

(2) C: 1.9477340410546757,  Accuracy: 0.8045416839485691

(3) C: 3.7936678946831774,  Accuracy: 0.8087239043273884

(4) C: 7.38905609893065,  Accuracy: 0.8098645098852482

(5) C: 14.391916095149892,  Accuracy: 0.810348403152219

(6) C: 28.031624894526125,  Accuracy: 0.8103138393474354

(7) C: 54.598150033144236,  Accuracy: 0.8097608184708973

(8) C: 106.34267539816545,  Accuracy: 0.8100373289091664

(9) C: 207.1272488898345,  Accuracy: 0.8100373289091664

(10) C: 403.4287934927351,  Accuracy: 0.8098299460804645



In [10]:
ind = acc_dev.index(max(acc_dev))
clf = LogisticRegression(penalty = 'l2', 
                         C = np.exp(np.linspace(0,6,10))[ind], 
                         solver = 'newton-cg', 
                         multi_class = 'multinomial', 
                         max_iter = 1000)

clf.fit(X_train, y_train)
print('test accuracy: {}'.format(accuracy_score(clf.predict(X_test), y_test)))
print('dev accuracy: {}'.format(accuracy_score(clf.predict(X_dev), y_dev)))

test accuracy: 0.8082572356313081
dev accuracy: 0.810348403152219


## Часть 3. Задание BERT (4 балла за 3 и 4 части)

Так как обучающих предложений очень мало, попробуем использовать модель BERT, предобученную на большом датасете. Будем использовать библиотеку transformers. Для обучения модели используйте данные до обработки из предыдущего пункта.

In [0]:
BERT_MODEL_NAME = "bert-base-uncased"
NUM_LABELS = len(set(y_train))

In [12]:
!pip install transformers #colab

In [13]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW, WarmupLinearSchedule

import torch
from torch.utils.data import DataLoader, Dataset

Модель BERT работает с специальным форматом данных — все токены из предложения получены с помощью алгоритма BPE. Класс BertTokenizer позволяет получить BPE разбиение для предложения.

In [0]:
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

В библиотеке transformers есть специальный класс для работы с задачей классификации — BertForSequenceClassification. Воспользуемся им, чтобы задать модель.

In [36]:
bert_model = BertForSequenceClassification.from_pretrained(
    BERT_MODEL_NAME, num_labels=NUM_LABELS
)

bert_model.to('cuda')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

Реализуем специальный кастомный датасет для токенизированных с помощью BPE предложений. Каждое предложение должно быть преобразовано в последовательность BPE индексов. Не забудьте, что в начале каждого предложения должен стоять специальный токен [CLS], а в конце должен стоять специальный токен [SEP].

Задайте датасет, используя BertTokenizer:

In [0]:
class BertTokenizedDataset(Dataset):
    def __init__(self, tokenizer, text_data, target_data=None, max_length=256):
        """
        Parameters
        ----------
        tokenizer : instance of BertTokenizer
        text_data : list of str
            List of input sentences
        target_data : list of int
            List of input targets
        max_length : int
            Maximum length of input sequence (length in bpe tokens)
        """
        super(BertTokenizedDataset, self).__init__()
        self.data = []
        self.target_data = target_data

        for sentence in text_data:
            tokens = tokenizer.encode('[CLS] ' + sentence + ' [SEP]', max_length=max_length)
            tokens = torch.LongTensor(tokens)
            self.data.append(tokens)
        
        
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        if self.target_data is not None:
            return self.data[i], self.target_data[i]
        else:
            return self.data[i]

Получите все датасеты для всех типов данных. 

**Замечание**. После получения есть смысл сохранить все датасеты на диск, т.к. предобработка занимает время.

In [0]:
train_dataset = BertTokenizedDataset(tokenizer, train_data, y_train)
dev_dataset = BertTokenizedDataset(tokenizer, dev_data, y_dev)
test_dataset = BertTokenizedDataset(tokenizer, test_data, y_test)

In [18]:
train_dataset.data[5]

tensor([  101, 12123,  8168,  2020,  4663,  1998, 20302, 23274,  2094,  2007,
         2051,  1011, 10395, 10047, 23041, 11253,  7630, 14604, 12589,  4632,
        22916, 12515,  1996, 12123,  3798,  1997,  4949, 22932,  1010, 16137,
         2361,  1011,  1015,  1010,  1998, 16137,  2361,  1011,  1017,  1012,
          102])

Используем  класс PadSequences, чтобы задать способ паддинга, работающий с встроенным в pytorch DataLoader.

In [0]:
class PadSequences:
    def __init__(self, use_labels=False):
        self.use_labels = use_labels
    
    def __call__(self, batch):
        """
        Parameters
        ----------
        batch : list of objects or list of (object, label)
            Each object is list of int indexes.
            Each label is int.
        """
        data_label_batch = batch if self.use_labels else [(x, 0) for x in batch]
            
        # Sort the batch in the descending order
        sorted_batch = sorted(data_label_batch, key=lambda x: x[0].shape[0], reverse=True)
        # Get each sequence and pad it
        sequences = [x[0] for x in sorted_batch]
        sequences_padded = torch.nn.utils.rnn.pad_sequence(sequences, batch_first=True)
        max_lenght = len(sequences[0])

        # Also need to store the length of each sequence
        # This is later needed in order to unpad the sequences
        lengths = torch.LongTensor([[1] * len(x) + [0] * (max_lenght - len(x)) for x in sequences])
        # Don't forget to grab the labels of the *sorted* batch
        
        if self.use_labels:
            labels = torch.LongTensor([x[1] for x in sorted_batch])
            return sequences_padded, lengths, labels
        else:
            return sequences_padded

Зададим DataLoader для каждого из датасетов:

In [0]:
BATCH_SIZE = 16

In [0]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                              collate_fn=PadSequences(use_labels=True))

dev_dataloader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False,
                              collate_fn=PadSequences(use_labels=True))

test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False,
                              collate_fn=PadSequences(use_labels=True))

Заметьте, что модель трансформера обучается по достаточному большому размеру батча (обычно 64), который скорее всего не будет влезать на вашу видеокарту. Поэтому, рекомендуется "накапливать" градиенты за несколько итераций. С помощью параметра ACCUMULATION_STEPS задайте, раз в сколько итераций вам необходимо делать шаг метода оптимизации.

In [0]:
EPOCH_AMOUNT = 2
TRAIN_LENGTH = len(train_dataset)
BATCH_SIZE = 16
ACCUMULATION_STEPS = 4

LR = 2e-5

Посчитайте общее число раз, когда ваш оптимизатор будет делать обновления на основе выбранных значений EPOCH_AMOUNT, BATCH_SIZE, ACCUMULATION_STEPS и  TRAIN_LENGTH. Эта величина нужна для правильного задания параметров оптимизаторов.

In [23]:
train_optimization_step_amount_float = EPOCH_AMOUNT * (TRAIN_LENGTH / (BATCH_SIZE * ACCUMULATION_STEPS ))
train_optimization_step_amount = EPOCH_AMOUNT * int(TRAIN_LENGTH / (BATCH_SIZE * ACCUMULATION_STEPS ))
print('train_optimization_step_amount = {} '.format(train_optimization_step_amount_float))
print('train_optimization_step_amount we use = {} '.format(train_optimization_step_amount))

train_optimization_step_amount = 921.65625 
train_optimization_step_amount we use = 920 


Зададим параметры оптимизаторов. Мы будем использовать специальные оптимизаторы из библиотеки transformers AdamW и WarmupLinearSchedule, обеспечивающие плавный разгон и медленное затухание темпа обучения.

In [0]:
optimizer = AdamW(bert_model.parameters(), lr=LR, correct_bias=False)
scheduler = WarmupLinearSchedule(
    optimizer,
    warmup_steps=train_optimization_step_amount * 0.05,
    t_total=train_optimization_step_amount,
)

Для некоторых групп параметров зададим коэффициенты регуляризации.

In [0]:
param_optimizer = list(bert_model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

## Часть 4. Обучение BERT 

Теперь всё готово к тому, чтобы дообучить BERT на датасете train_dataset!

Используйте dev_dataset для выбора гиперпараметров модели и обучения. Задание будет засчтано на полный балл если на dev_dataset и test_dataset точность будет выше 0.84.

In [33]:
torch.cuda.empty_cache()
!nvidia-smi

Mon Nov 18 16:57:37 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    42W / 250W |   2941MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [39]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [40]:
bert_model.train()
step = 0

for epoch in range(EPOCH_AMOUNT):
    for iteration, batch in enumerate(train_dataloader):
        input_ids, labels_ids = batch[0], batch[2]
        outputs = bert_model(input_ids.to(device), labels=labels_ids.to(device))
        loss, predict = outputs
        loss.backward()

        if (iteration + 1) % ACCUMULATION_STEPS == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            step += 1
        if iteration % 400 == 0:
            print('Train: Epoch {0}/{1}, Step : {2}/{3}, Loss: {4} \n'.format(epoch + 1, EPOCH_AMOUNT, step, train_optimization_step_amount, loss))
        
      
    bert_model.eval()
    num_steps = 0
    ratio_right_ans_dev = 0

    for batch in dev_dataloader:
        input_ids, labels_ids = batch[0], batch[2]
        input_ids, labels_ids = input_ids.to(device), labels_ids.to(device)
        labels_ids = labels_ids.to('cpu').numpy()

        with torch.no_grad():
            predict = bert_model(input_ids)  
        predict = predict[0].to('cpu').numpy()
        predict = np.argmax(predict, axis=1)

        ratio_right_ans_dev += np.sum(predict == labels_ids)/ len(labels_ids)
        num_steps += 1

    print('Dev accuracy: {}'.format(ratio_right_ans_dev /  num_steps))

Train: Epoch 1/2, Step : 0/920, Loss: 1.6729981899261475 

Train: Epoch 1/2, Step : 100/920, Loss: 0.4701715111732483 

Train: Epoch 1/2, Step : 200/920, Loss: 0.7067486643791199 

Train: Epoch 1/2, Step : 300/920, Loss: 0.7015130519866943 

Train: Epoch 1/2, Step : 400/920, Loss: 0.2623586654663086 

Dev accuracy: 0.8609038142620232
Train: Epoch 2/2, Step : 461/920, Loss: 0.6440114974975586 

Train: Epoch 2/2, Step : 561/920, Loss: 0.4651113748550415 

Train: Epoch 2/2, Step : 661/920, Loss: 0.29276227951049805 

Train: Epoch 2/2, Step : 761/920, Loss: 0.11788536608219147 

Train: Epoch 2/2, Step : 861/920, Loss: 0.41912785172462463 

Dev accuracy: 0.8648770038695411


In [41]:
bert_model.eval()
test_num_steps = 0
ratio_right_ans_test = 0

for batch in test_dataloader:
    input_ids, labels_ids = batch[0], batch[2]
    input_ids, labels_ids = input_ids.to(device), labels_ids.to(device)
    labels_ids = labels_ids.detach().to('cpu').numpy()

    with torch.no_grad():
        predict = bert_model(input_ids)  
    predict = predict[0].to('cpu').numpy()
    predict = np.argmax(predict, axis=1)
    
    ratio_right_ans_test += np.sum(predict == labels_ids) / len(labels_ids)
    test_num_steps += 1

print('Test accuracy: {}'.format(ratio_right_ans_test /  test_num_steps))

Test accuracy: 0.8613266597899475


In [42]:
print('Dev accuracy: {}'.format(ratio_right_ans_dev /  num_steps))
print('Test accuracy: {}'.format(ratio_right_ans_test /  test_num_steps))

Dev accuracy: 0.8648770038695411
Test accuracy: 0.8613266597899475


## Бонусная часть (до 3 баллов)

Улучшите качество (на обеих выборках), используя любые способы (кроме использования дополнительных обучающих данных датасета RCT2000):

* $> 0.86$ — 1 балл 
* $> 0.88$ — 2 балла
* $> 0.9$ — 3 балла

In [43]:
print('Dev accuracy: {}'.format(ratio_right_ans_dev /  num_steps))
print('Test accuracy: {}'.format(ratio_right_ans_test /  test_num_steps))

Dev accuracy: 0.8648770038695411
Test accuracy: 0.8613266597899475
